# Calculate Classification metrics
A ready-to-use script to find classification metrics for class pairs

**Input**:
- Existing project labeled with both ground truth and predicted categories encoded as tags.
- At least one pair of corresponding ground truth and prediction classes, e.g. ("car_gt", "car_predicted").

**Output**:
- Classification metrics for each class pair

## Imports

In [1]:
import supervisely_lib as sly
import os
import collections
from prettytable import PrettyTable
from tqdm import tqdm

## Configuration

Edit the following settings for your own case

In [2]:
# Change this field to the name of your team, where target workspace exists.
team_name = "jupyter_tutorials"

# Change this field to the of your workspace, where target project exists.
workspace_name = "metrics_tutorials"

# Change this field to the name of your target project.
project_name = "classification_metrics_demo_project"

# Configure the following dictionary so that is will match pairs of ground truth and predicted classes
# for which the metrics will be caluclated.
tags_mapping = {
    "kiwi": "kiwi_pred",
    "lemon": "lemon_pred",    
}

# If you are running this notebook on a Supervisely web instance, the connection
# Edit those values if you run this notebook on your own PC
# details below will be filled in from environment variables automatically.
#
# If you are running this notebook locally on your own machine, edit to fill in the
# connection details manually. You can find your access token at
# "Your name on the top right" -> "Account settings" -> "API token".
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

## Script setup

Import nessesary packages and initialize Supervisely API to remotely manage your projects

In [3]:
# Initialize API object
api = sly.Api(address, token)

## Verify input values

Test that context (team / workspace / project) exists

In [4]:
team = api.team.get_info_by_name(team_name)
if team is None:
    raise RuntimeError("Team {!r} not found".format(team_name))

workspace = api.workspace.get_info_by_name(team.id, workspace_name)
if workspace is None:
    raise RuntimeError("Workspace {!r} not found".format(workspace_name))
    
project = api.project.get_info_by_name(workspace.id, project_name)
if project is None:
    raise RuntimeError("Project {!r} not found".format(project_name))
    
print("Team: id={}, name={}".format(team.id, team.name))
print("Workspace: id={}, name={}".format(workspace.id, workspace.name))
print("Project: id={}, name={}".format(project.id, project.name))

Team: id=5, name=antonc
Workspace: id=22, name=metrics
Project: id=86, name=classification_metrics_demo_project


## Get source project meta

Project meta contains information about classes and tags.

In [5]:
meta_json = api.project.get_meta(project.id)
meta = sly.ProjectMeta.from_json(meta_json)

# check that all classes exist
project_tags_names = list(tags_mapping.keys()) + list(tags_mapping.values())

for tag_name in project_tags_names:
    if not meta.tag_metas.has_key(tag_name):
        raise RuntimeError("Class {!r} not found in source project {!r}".format(class_name, project.name))

## Create metric evaluator

In [6]:
classification_evaluator = sly.ClassificationMetrics(tags_mapping)

## Iterate over all images, and calculate metric by annotations pairs

In [7]:
for dataset in api.dataset.get_list(project.id):
    
    # generate dataset name in destination project if it exists
    print("Processing: project = {!r}, dataset = {!r}".format(project.name, dataset.name), flush=True)
    
    images = api.image.get_list(dataset.id)
    with tqdm(total=len(images), desc="Process annotations") as progress_bar:
        for batch in sly.batched(images):
            image_ids = [image_info.id for image_info in batch]
            ann_infos = api.annotation.download_batch(dataset.id, image_ids)
            
            for ann_info in ann_infos:
                ann = sly.Annotation.from_json(ann_info.annotation, meta)
                # We are using the same annotation on the both side of the metric computation
                # (classes_mapping provides the corresponding classes that we will look for
                # in the annotation), but it is also possible to use different annotations
                # on left and right, e.g. to compare the source hand-labeled project to a
                # neural netork inference result.
                classification_evaluator.add_pair(ann, ann)
            
            progress_bar.update(len(batch))

Processing: project = 'classification_metrics_demo_project', dataset = 'ds1'


Process annotations: 100%|██████████| 6/6 [00:00<00:00, 155.42it/s]


## Print results with default logger

The results are logged with the default Supervisely logger, so that the same code can be used in any custom plugin, and the log output would be nicely formatted in the task log.

In [8]:
classification_evaluator.log_total_metrics()

{"message": "                                                                                ", "timestamp": "2019-04-24T12:03:04.930Z", "level": "info"}
{"message": "********************************************************************************", "timestamp": "2019-04-24T12:03:04.931Z", "level": "info"}
{"message": "P = condition positive (the number of real positive cases in the data)          ", "timestamp": "2019-04-24T12:03:04.932Z", "level": "info"}
{"message": "N = condition negative (the number of real negative cases in the data)          ", "timestamp": "2019-04-24T12:03:04.933Z", "level": "info"}
{"message": "TP = True Positive prediction                                                   ", "timestamp": "2019-04-24T12:03:04.933Z", "level": "info"}
{"message": "TN = True Negative prediction                                                   ", "timestamp": "2019-04-24T12:03:04.934Z", "level": "info"}
{"message": "FP = False Positive prediction (Type I error)                  

## Print results manually

In [9]:
# Metrics for each pair of classes separately.
per_class_metrics = classification_evaluator.get_metrics()

# Metrics aggregated over all pairs of classes from classes_mapping
total_metrics = classification_evaluator.get_total_metrics()

table = PrettyTable(["classes pair", "metrics values"])

def build_values_text(values):
    return ''.join(
        "{}: {}\n".format(metrics_name, value)
        for metrics_name, value in values.items())
    
for gt_class, metric_values in per_class_metrics.items():
    pair_text = "{} <-> {}".format(gt_class, tags_mapping[gt_class])
    table.add_row([pair_text, build_values_text(metric_values)])

table.add_row(["TOTAL", build_values_text(total_metrics)])
print(table.get_string())

+----------------------+--------------------------------+
|     classes pair     |         metrics values         |
+----------------------+--------------------------------+
|  kiwi <-> kiwi_pred  |        true-positive: 3        |
|                      |        true-negative: 2        |
|                      |       false-positive: 0        |
|                      |       false-negative: 1        |
|                      |  accuracy: 0.8333333333333334  |
|                      |         precision: 1.0         |
|                      |          recall: 0.75          |
|                      | F1-measure: 0.8571428571428571 |
|                      |                                |
| lemon <-> lemon_pred |        true-positive: 2        |
|                      |        true-negative: 3        |
|                      |       false-positive: 1        |
|                      |       false-negative: 0        |
|                      |  accuracy: 0.8333333333333334  |
|             

# Done!